In [1]:
import numpy as np

1. Мы рекурсивно спускаемся по дереву, а потом возвращаемся по этой рекурсии с уже найденным списком соседей. 

2. Затем сравниваем расстояние до самого дальнего из соседей с расстоянием до разделяющей границы на этом уровне и,
3. если оно больше, то делаем запрос к соседнему поддереву, 
4. получаем второй список соседей и объединяем два списка (новый и старый)

In [200]:
class Node:
    def __init__(self, median=None, axis=None, data=None):
        self.median = median
        self.axis = axis
        self.data = data
        
        self.left = None
        self.right = None
    
    def is_leaf(self):
        return self.left is None and self.right is None
    
    def neighbors(self, point, k=2):
        dist = np.sqrt(np.sum((self.data[:, 1:] - point)**2, axis=1))
        idxs = np.argsort(dist)[:k]
        
        return self.data[idxs, 0], dist[idxs]        

In [201]:
# TODO: not working!
def merge(n1, d1, n2, d2, k):
    dists = np.hstack([d1, d2])
    neighs = np.hstack([n1, n2])
    
    idxs = np.argsort(dists)[:k]
    
    return neighs[idxs], dists[idxs]

In [202]:
class KDTree:
    def __init__(self, X, leaf_size=10):
        self.X = np.hstack([np.arange(X.shape[0]).reshape(-1, 1), X])
        self.dim = X.shape[1]
        
        self.leaf_size = leaf_size
        self.root = self._build_tree(self.X)
        
    def _build_tree(self, X, depth=0):
        axis = (depth % self.dim) + 1
        
        median = np.median(X[:, axis])
        X_left, X_right = X[X[:, axis] < median], X[X[:, axis] >= median]
        
        if X_left.shape[0] < self.leaf_size or X_right.shape[0] < self.leaf_size:
            return Node(data=X)
        
        root = Node(median=median, axis=axis)
        root.left = self._build_tree(X_left, depth + 1)
        root.right = self._build_tree(X_right, depth + 1)
        
        return root
    
    def _query(self, root, point, k):
        if root.is_leaf():
            return root.neighbors(point, k)
        
        axis = root.axis
        if point[axis] < root.median:
            neigh, dist = self._query(root.left, point, k)
            other_node = root.right
        else:
            neigh, dist = self._query(root.right, point, k)
            other_node = root.left
            
        if dist[-1] > abs(point[axis] - root.median):
            new_neigh, new_dist = self._query(other_node, point, k)
            
            return merge(neigh, dist, new_neigh, new_dist, k)
            
        return neigh, dist
    
    def query(self, X, k):
        neighs = np.zeros(shape=(X.shape[0], k))
        
        for i, point in enumerate(X):
            neigh, dist = self._query(self.root, point, k)
            neighs[i, :] = neigh
        
        return neighs

In [203]:
X = np.random.uniform(size=(10, 3))

In [204]:
tree = KDTree(X)

In [205]:
tree.query(X[:5], k=10)

array([[0., 1., 8., 2., 5., 6., 9., 3., 4., 7.],
       [1., 0., 8., 2., 5., 4., 3., 6., 9., 7.],
       [2., 8., 4., 1., 5., 9., 7., 3., 0., 6.],
       [3., 5., 2., 6., 4., 9., 1., 7., 8., 0.],
       [4., 2., 8., 3., 1., 5., 7., 9., 0., 6.]])